# Titanic - Machine Learning from Disaster

## Data Preparation

In [1]:
import pandas as pd

train_df = pd.read_csv('train.csv')
train_df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [2]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Separate the features and the label.

In [3]:
y = train_df['Survived']
X = train_df.drop('Survived', axis=1)

## Data Wrangling

It seems that *PassengerId*, *Name*, and *Ticket* would not have any relationship with our later predictions. So drop these.

In addition, *Pclass* already contains which class cabin the passengers were in, so drop *Cabin* as well.

In [4]:
X.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

Remove null entries from *Age* and *Embared* columns. We will fill *Age* with its mean and the others with 'N'.

In [5]:
X['Age'].fillna(X['Age'].mean(), inplace=True)
X['Embarked'].fillna('N', inplace=True)

Convert string columns into numerical values.

In [6]:
from sklearn.preprocessing import LabelEncoder

string_features = ['Sex', 'Embarked']
for feature in string_features:
    label_encoder = LabelEncoder()
    X[feature] = label_encoder.fit_transform(X[feature])

Split the dataset into train and test parts.

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Train Model

We are going to use decision tree classifier. Hyperparameters are determined by grid search.

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [2, 3, 5, 10],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1, 5, 8]
}
decision_tree_classifier = DecisionTreeClassifier()
grid_search_cv = GridSearchCV(decision_tree_classifier, param_grid=param_grid, scoring='accuracy', cv=5)

grid_search_cv.fit(X_train, y_train)

estimator = grid_search_cv.best_estimator_

## Test Model

In [9]:
from sklearn.metrics import accuracy_score

predictions = estimator.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy on the test dataset: {accuracy:.4f}')

Accuracy on the test dataset: 0.8492
